In [1]:
import numpy as np

from GF16 import GFT, sbox, sboxinv

In [2]:
def make_bin(n):
    n = bin(n)
    n = n.replace('0b', '')
    while(len(n) < 4):
        n = "0" + n
    n = list(n)
    n = [int(i) for i in n]
    return n

In [3]:
def RotNib(w):
    w = np.asarray(w)
    w = np.roll(w, 4)
    w = w.tolist()
    return w

In [4]:
def SubNib(w):
    a = []
    l = len(w)
    for i in range(0,l,4):
        n = w[i:i+4]
        n = ''.join(map(str,n))
        a = a + make_bin(sbox[int(n,2)])
    return a

In [5]:
def ShRow(w):
    for i in range(4, 8):
        w[i], w[i+8] = w[i+8], w[i]
    return w

In [6]:
def XOR(a, b):
    a = np.asarray(a)
    b = np.asarray(b)
    c = np.logical_xor(a, b)
    c = c.astype(int)
    c = c.tolist()
    return c

In [7]:
def generate_keys(K):
    l = int(len(K)/2)
    w0 = K[:l]
    w1 = K[l:]
    w2 = XOR(XOR(w0, [1,0,0,0,0,0,0,0]), SubNib(RotNib(w1)))
    w3 = XOR(w2, w1)
    w4 = XOR(XOR(w2, [0,0,1,1,0,0,0,0]), SubNib(RotNib(w3)))
    w5 = XOR(w4, w3)
    key0 = w0+w1
    key1 = w2+w3
    key2 = w4+w5
    return key0, key1, key2

In [8]:
def gen_S(x):
    s = []
    l = len(x)
    for i in range(0,l,4):
        n = x[i:i+4]
        n = ''.join(map(str,n))
        s.append(int(n,2))
    s = [[s[0], s[1]], [s[2], s[3]]]
    s = np.asarray(s)
    s = s.transpose()
    return s

In [9]:
def mat_mul(s):
    result = [[0,0], [0,0]]
    result[0][0] = GFT[s[0][0]][1] ^ GFT[s[0][1]][4]
    result[0][1] = GFT[s[0][0]][4] ^ GFT[s[0][1]][1]
    result[1][0] = GFT[s[1][0]][1] ^ GFT[s[1][1]][4]    
    result[1][1] = GFT[s[1][0]][4] ^ GFT[s[1][1]][1]
    result = sum(result,[])
    result = [make_bin(i) for i in result]
    result = sum(result, [])
    return result

In [10]:
def encrypt(P, K):
    key0, key1, key2 = generate_keys(K)
    # Add Round 0 key
    R0 = XOR(P, key0)
    # Round 1
    x = ShRow(SubNib(R0))
    s = gen_S(x)
    prod = mat_mul(s)
    res = XOR(prod, key1)
    # Final Round
    ciphertext = XOR(ShRow(SubNib(res)), key2)
    return ciphertext

In [11]:
def SubNibd(w):
    a = []
    l = len(w)
    for i in range(0,l,4):
        n = w[i:i+4]
        n = ''.join(map(str,n))
        a = a + make_bin(sboxinv[int(n,2)])
    return a

In [12]:
def mat_mul_inv(s):
    result = [[0,0], [0,0]]
    result[0][0] = GFT[9][s[0][0]] ^ GFT[2][s[1][0]]
    result[0][1] = GFT[9][s[0][1]] ^ GFT[2][s[1][1]]
    result[1][0] = GFT[2][s[0][0]] ^ GFT[9][s[1][0]]    
    result[1][1] = GFT[2][s[0][1]] ^ GFT[9][s[1][1]]
    result = sum(result, [])
    result = [make_bin(i) for i in result]
    result = sum(result, [])
    return result

In [13]:
def decrypt(C, K):
    key0, key1, key2 = generate_keys(K)
    s = XOR(SubNibd(ShRow(XOR(C,key2))), key1)
    s = gen_S(s)
    prod = mat_mul_inv(s)
    decipher = XOR(SubNibd(ShRow(prod)),key0)
    return decipher

In [14]:
plaintext = [1,1,0,1,0,1,1,1,0,0,1,0,1,0,0,0]
key = [0,1,0,0,1,0,1,0,1,1,1,1,0,1,0,1]
print("Plaintext: \t", plaintext)
print("Original Key: \t", key)
key0, key1, key2 = generate_keys(key)
print("\nGenerated Keys: ")
print("Key 0: ", key0)
print("Key 1: ", key1)
print("Key 2: ", key2)
ciphertext = encrypt(plaintext, key)
print("\nEncrypted text: ", ciphertext)
decipher = decrypt(ciphertext, key)
print("Decrypted text: ", decipher)

if(decipher == plaintext):
    print("\nS-AES Successful")
else:
    print("\nS-AES Failed")

Plaintext: 	 [1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]
Original Key: 	 [0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1]

Generated Keys: 
Key 0:  [0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1]
Key 1:  [1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0]
Key 2:  [1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]

Encrypted text:  [0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0]
Decrypted text:  [1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]

S-AES Successful


In [22]:
# Testing (Successfully passed all tests)
'''
for i in range(0, 10000):
    key = np.random.randint(2, size=16)
    key = key.tolist()
    plaintext = np.random.randint(2, size=16)
    plaintext = plaintext.tolist()
    key0, key1, key2 = generate_keys(key)
    ciphertext = encrypt(plaintext, key)
    decipher = decrypt(ciphertext, key)
    if(decipher == plaintext):
        print("S-AES Successful")
    else:
        print("S-AES Failed")
'''

'\nfor i in range(0, 10000):\n    key = np.random.randint(2, size=16)\n    key = key.tolist()\n    plaintext = np.random.randint(2, size=16)\n    plaintext = plaintext.tolist()\n    key0, key1, key2 = generate_keys(key)\n    ciphertext = encrypt(plaintext, key)\n    decipher = decrypt(ciphertext, key)\n    if(decipher == plaintext):\n        print("S-AES Successful")\n    else:\n        print("S-AES Failed")\n'